In [1]:
import random
import numpy as np
import pandas as pd
import csv
from tqdm import tqdm
import seaborn as sns
import os
import networkx as nx
import pdb
import pickle
from collections import Counter
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.feature_extraction import text as fe
from sklearn.decomposition import NMF, LatentDirichletAllocation
import matplotlib.pyplot as plt
import datetime
import time
import keras
import lightgbm
import spacy
from nltk import wordpunct_tokenize
from nltk.corpus import stopwords
import nltk
from hyperopt import STATUS_OK, Trials, hp, space_eval, tpe, fmin

Using TensorFlow backend.


In [2]:
with open(r"training.txt", "r") as f:
    reader = csv.reader(f)
    training  = list(reader)
training = [element[0].split(" ") for element in training]
training = pd.DataFrame(training, columns=['Node1', 'Node2', 'Link'])
print("Training examples shape: {}".format(training.shape))

with open(r"testing.txt", "r") as f:
    reader = csv.reader(f)
    testing  = list(reader)
testing = [element[0].split(" ") for element in testing]
testing = pd.DataFrame(testing, columns=['Node1', 'Node2'])
print("Testing examples shape: {}".format(testing.shape))

Training examples shape: (453797, 3)
Testing examples shape: (113450, 2)


In [3]:
'''
uncomment lines for reduced corpus with stopword removal. In future integrate stemmer here, multi-language
'''
NODE_INFO_DIRECTORY = r"node_information/text/"

corpus_path = r"pickles/simple_corpus.PICKLE" 
ids_path = r"pickles/IDs.PICKLE"
if os.path.exists(corpus_path):
    with open(corpus_path, 'rb') as f:
        corpus = pickle.load(f)
    f.close()
    with open(ids_path, 'rb') as f:
        ids = pickle.load(f)
    f.close()
else:
    corpus = []
    ids = []
    for filename in tqdm(os.listdir(NODE_INFO_DIRECTORY)):
        with open(NODE_INFO_DIRECTORY + filename, 'r', encoding='UTF-8', errors='ignore') as f:
            doc_string = []
            for line in f:
                [doc_string.append(token) for token in line.lower().strip().split(" ")]
            corpus.append(' '.join(doc_string))
            ids.append(filename[:-4])
    with open(corpus_path, '+wb') as f:
        pickle.dump(corpus, f)
    f.close()
    with open(ids_path, '+wb') as f:
        pickle.dump(ids, f)
    f.close() 

# in order of alphabetical text information i.e. 0, 1, 10, 100
node_info = pd.DataFrame({'ID': ids, 'Corpus': corpus})
node_info_ID = node_info.set_index(['ID'])
print("Training node info shape: {}".format(node_info.shape))

Training node info shape: (33226, 2)


In [4]:
train_graph_split_path = 'pickles/train_graph_split.PICKLE'

if os.path.exists(train_graph_split_path):
    with open(train_graph_split_path, 'rb') as f:
        keep_indices = pickle.load(f)
    f.close()
else:
    keep_indices = random.sample(range(len(training)), k=int(len(training) * 0.05))
    with open(train_graph_split_path, '+wb') as f:
        pickle.dump(keep_indices, f)
    f.close()

data_train_val = training.iloc[keep_indices]
data_train = training.loc[~training.index.isin(keep_indices)]

In [5]:
linked_nodes = data_train.loc[data_train['Link']=='1']
linked_nodes = linked_nodes[['Node1', 'Node2']]
linked_nodes.to_csv('linked_nodes.txt', sep=' ', index=False, header=False)

In [6]:
G=nx.read_edgelist('linked_nodes.txt', create_using=nx.Graph(), nodetype = str)

In [ ]:
'''
for i in tqdm(list(node_info.index)):
    if node_info.loc[i]['ID'] in G:
        G.node[node_info.loc[i]['ID']]['T'] = node_info.loc[i]['Corpus']
'''

## Training

In [ ]:
katz_nd = nx.katz_centrality_numpy(G, alpha=0.1, beta=1.0)

In [ ]:
def katz_centrality(index, dataset):
    node1 = dataset['Node1'][index]
    node2 = dataset['Node2'][index]
    if (node1 in katz_nd.keys()) and (node2 in katz_nd.keys()): 
        katz = (katz_nd[node1] + katz_nd[node2])/2
        return katz
    else: 
        return 0

perf_val_data['page_rank']=list(map(lambda i:katz_centrality(i, training), tqdm(perf_val_data['original_index'])))
perf_test_data['page_rank']=list(map(lambda i:katz_centrality(i, testing), tqdm(perf_test_data['original_index'])))

In [ ]:
#switch to fit_transform for tfidf

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.cluster import KMeans, MiniBatchKMeans

print("Performing dimensionality reduction using LSA")
svd = TruncatedSVD(800)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)

X = lsa.fit_transform(corpus_tfidf_matrix)

explained_variance = svd.explained_variance_ratio_.sum()
print("Explained variance of the SVD step: {}%".format(
    int(explained_variance * 100)))

In [ ]:
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn import metrics

if opts.minibatch:
    km = MiniBatchKMeans(n_clusters=true_k, init='k-means++', n_init=1,
                         init_size=1000, batch_size=1000, verbose=opts.verbose)
else:
    km = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1,
                verbose=opts.verbose)

print("Clustering sparse data with %s" % km)
t0 = time()
km.fit(X)
print("done in %0.3fs" % (time() - t0))
print()

print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels, km.labels_))
print("Completeness: %0.3f" % metrics.completeness_score(labels, km.labels_))
print("V-measure: %0.3f" % metrics.v_measure_score(labels, km.labels_))
print("Adjusted Rand-Index: %.3f"
      % metrics.adjusted_rand_score(labels, km.labels_))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, km.labels_, sample_size=1000))
print()

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2
n_k = 1000
print("Extracting %d best features by a chi-squared test" %
      n_k)
ch2 = SelectKBest(chi2, k=n_k)
X_train = ch2.fit_transform(X_train, y_train)
X_test = ch2.transform(X_test)

## Node2Vec Experiments

In [11]:
import networkx as nx
from node2vec import Node2Vec

In [8]:
# Create a graph
graph = G

In [10]:
node2vec_path = r"'pickles/NODE2VEC.PICKLE'"
if os.path.exists(node2vec_path):
    print('Node2Vec already calculated')
else:
    # Precompute probabilities and generate walks - **ON WINDOWS ONLY WORKS WITH workers=1**
    node2vec = Node2Vec(graph, dimensions=64, walk_length=30, num_walks=200, workers=1)


Computing transition probabilities:   0%|                                        | 103/33139 [00:24<1:57:06,  4.70it/s]


Computing transition probabilities:   1%|▎                                       | 217/33139 [00:41<1:21:28,  6.73it/s]


Computing transition probabilities:   1%|▍                                       | 360/33139 [00:59<1:14:34,  7.33it/s]


Computing transition probabilities:   1%|▌                                       | 493/33139 [01:16<1:49:51,  4.95it/s]


Computing transition probabilities:   2%|▊                                       | 628/33139 [01:32<1:09:43,  7.77it/s]


...

Computing transition probabilities: 100%|███████████████████████████████████████▉| 33048/33139 [26:55<00:02, 35.55it/s]


Computing transition probabilities: 100%|████████████████████████████████████████| 33139/33139 [26:57<00:00, 20.49it/s]

Generating walks (CPU: 1):  27%|█████████████▏                                   | 54/200 [1:54:39<5:11:51, 128.16s/it]


Generating walks (CPU: 1):  61%|█████████████████████████████▎                  | 122/200 [4:21:18<2:46:50, 128.34s/it]


Generating walks (CPU: 1):  95%|███████████████████████████████████████████████▌  | 190/200 [6:50:42<21:24, 128.49s/it]


Generating walks (CPU: 1): 100%|██████████████████████████████████████████████████| 200/200 [7:14:31<00:00, 130.36s/it]


In [25]:
from gensim.models import Word2Vec

node2vec_model_path = r"pickles/EMBEDDING_MODEL_FILENAME.model"
if os.path.exists(node2vec_model_path):
    model = Word2Vec.load(node2vec_model_path)
else:
    # Embed nodes
    model = node2vec.fit(window=10, min_count=1, batch_words=4)  # Any keywords acceptable by gensim.Word2Vec can be passed
    # Save embeddings for later use
    model.wv.save_word2vec_format('pickles/EMBEDDING_FILENAME.embed')
    # Save model for later use
    model.save('pickles/EMBEDDING_MODEL_FILENAME.model')

In [15]:
# Embed edges using Hadamard method
from node2vec.edges import HadamardEmbedder
edges_embs = HadamardEmbedder(keyed_vectors=model.wv)

In [49]:
embed_size = len(edges_embs[('0', '1')])
df_train = pd.DataFrame(0, index=np.arange(len(data_train_val)), columns=range(embed_size))
j = []
for j, i in tqdm(enumerate(data_train_val.index), position=0, leave=True, total = len(data_train_val)):
    try:
        df_train.loc[j] = edges_embs[(data_train_val.loc[i]['Node1'], data_train_val.loc[i]['Node2'])]
    except:
        df_train.loc[j] = np.zeros(embed_size)

100%|███████████████████████████████████████████████████████████████████████████| 22689/22689 [01:23<00:00, 272.36it/s]


In [ ]:
embed_size = len(edges_embs[('0', '1')])
df_test = pd.DataFrame(0, index=np.arange(len(testing)), columns=range(embed_size))
j = []
for i in tqdm(range(len(testing)), position=0, leave=True):
    try:
        df_test.loc[i] = edges_embs[(testing.loc[i]['Node1'], testing.loc[i]['Node2'])]
    except:
        df_test.loc[i] = np.zeros(embed_size)

 47%|██████████████████████████████████▋                                       | 53106/113450 [01:49<02:24, 418.81it/s]

In [99]:
X = df # df_train in the future
y = data_train_val['Link']
y = list(map(lambda i: int(i), y))

X_train, X_test, y_train, y_test = train_test_split(normalized, y, test_size=0.15, random_state=1)

In [100]:
lgbm = lightgbm.LGBMClassifier()
model_lgbm = lgbm.fit(X_train, y_train)
predictions = model_lgbm.predict(X_test)
print(f1_score(y_test, predictions))

0.9081105369579869


In [ ]:
lgbm = lightgbm.LGBMClassifier()
model_lgbm = lgbm.fit(X, y)
predictions = model_lgbm.predict(df_test)

In [ ]:
final_pred = pd.concat([np.arange(len(df_test)), pd.DataFrame(predictions)],axis=1)
final_pred.columns = ['id', 'predicted']
with open("node2vec_prediction.csv","+w") as pred:
    csv_out = csv.writer(pred)
    csv_out.writerow(['id','predicted'])
    for index, row in final_pred.iterrows():
        csv_out.writerow(row)

## Keras Experiments

In [85]:
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization

scaler = StandardScaler()
normalized = scaler.fit_transform(X)

In [87]:
from keras import backend as K
from keras import callbacks
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def create_model():
    model = Sequential()
    model.add(Dense(20, input_dim=64, activation='relu'))
    BatchNormalization()
    model.add(Dense(20, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', f1_m])
    return(model)

multi_rnn = create_model()

In [88]:
es = callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1)
history = multi_rnn.fit(normalized, y, epochs = 100, batch_size = 50, verbose = 2, validation_split=0.15, callbacks=[es])

Train on 19285 samples, validate on 3404 samples
Epoch 1/100
 - 8s - loss: 0.3834 - acc: 0.8324 - f1_m: 0.8452 - val_loss: 0.3428 - val_acc: 0.8499 - val_f1_m: 0.8451
Epoch 2/100
 - 7s - loss: 0.3168 - acc: 0.8708 - f1_m: 0.8711 - val_loss: 0.3243 - val_acc: 0.8622 - val_f1_m: 0.8629
Epoch 3/100
 - 7s - loss: 0.2981 - acc: 0.8799 - f1_m: 0.8830 - val_loss: 0.3226 - val_acc: 0.8631 - val_f1_m: 0.8638
Epoch 4/100
 - 7s - loss: 0.2891 - acc: 0.8835 - f1_m: 0.8828 - val_loss: 0.3209 - val_acc: 0.8602 - val_f1_m: 0.8599
Epoch 5/100
 - 6s - loss: 0.2822 - acc: 0.8856 - f1_m: 0.8871 - val_loss: 0.3292 - val_acc: 0.8552 - val_f1_m: 0.8519
Epoch 6/100
 - 8s - loss: 0.2769 - acc: 0.8883 - f1_m: 0.8864 - val_loss: 0.3253 - val_acc: 0.8608 - val_f1_m: 0.8602
Epoch 7/100
 - 8s - loss: 0.2721 - acc: 0.8901 - f1_m: 0.8892 - val_loss: 0.3280 - val_acc: 0.8590 - val_f1_m: 0.8591
Epoch 8/100
 - 7s - loss: 0.2677 - acc: 0.8947 - f1_m: 0.8973 - val_loss: 0.3346 - val_acc: 0.8608 - val_f1_m: 0.8601
Epoch 9